In [2]:

# ===============================================================
# 📥 NE PAS EXECUTER C'EST UN FICHIER QUI A DEJA été EXECUTER
# ===============================================================







import pandas as pd
import numpy as np

# ===============================================================
# 📥 Lecture du fichier source
# ===============================================================
nom_fichier = 'statistiques_communes_dvf_2020_2024.csv'
stats = pd.read_csv(nom_fichier, sep=';', low_memory=False)

# Nettoyage de base
stats = stats.dropna(subset=['nom_commune', 'annee'])
stats['annee'] = pd.to_numeric(stats['annee'], errors='coerce')
stats = stats.dropna(subset=['annee'])
stats['annee'] = stats['annee'].astype(int)

# ===============================================================
# 🏙️ Détection et fusion des arrondissements (Paris, Lyon, Marseille)
# ===============================================================
def ville_simplifiee(nom):
    nom = str(nom)
    if nom.lower().startswith("paris"):
        return "Paris"
    elif nom.lower().startswith("lyon"):
        return "Lyon"
    elif nom.lower().startswith("marseille"):
        return "Marseille"
    else:
        return nom

stats['ville'] = stats['nom_commune'].apply(ville_simplifiee)

# On détecte les colonnes numériques (pour les agréger)
colonnes_numeriques = stats.select_dtypes(include=[np.number]).columns.tolist()
colonnes_non_num = [c for c in stats.columns if c not in colonnes_numeriques]

# Agrégation par ville et année :
# - Pour les colonnes numériques → somme (ou moyenne pondérée si applicable)
# - Pour les colonnes non numériques → première valeur (logique pour une donnée descriptive)
def aggreg_flexible(df):
    data = {}
    for col in df.columns:
        if col in colonnes_numeriques:
            if col == 'prix_m2_moyen' and 'nb_transactions' in df.columns:
                data[col] = (df['prix_m2_moyen'] * df['nb_transactions']).sum() / df['nb_transactions'].sum()
            else:
                data[col] = df[col].sum()
        elif col == 'nom_commune':
            data[col] = df['ville'].iloc[0]  # remplace par "Paris", "Lyon" ou "Marseille"
        else:
            data[col] = df[col].dropna().iloc[0] if not df[col].dropna().empty else np.nan
    return pd.Series(data)

agg = (
    stats.groupby(['ville', 'annee'], dropna=False)
    .apply(aggreg_flexible)
    .reset_index(drop=True)
)

# ===============================================================
# 🗺️ Sélection des grandes villes françaises
# ===============================================================
villes_selection = [
    'Paris', 'Lyon', 'Marseille',
    'Toulouse', 'Bordeaux', 'Montpellier',
    'Lille', 'Rennes', 'Strasbourg', 'Grenoble',
]

# On garde uniquement les villes voulues (insensible à la casse)
stats_simplifie = agg[agg['ville'].str.lower().isin([v.lower() for v in villes_selection])].copy()

# ===============================================================
# 💾 Enregistrement du CSV simplifié
# ===============================================================
fichier_sortie = 'statistiques_grandes_villes_2020_2024_toutes_colonnes.csv'
stats_simplifie.to_csv(fichier_sortie, sep=';', index=False, encoding='utf-8-sig')

print(f"✅ Fichier simplifié enregistré : {fichier_sortie}")
print(f"Nombre de lignes : {len(stats_simplifie)}")
print(stats_simplifie[['ville', 'annee', 'prix_m2_moyen', 'nb_transactions']].head())

KeyboardInterrupt: 

In [ ]:
#TRAITEMENT LOYER VILLE
warnings.filterwarnings('ignore')

# ============================================================================
# SECTION 1 : CHARGEMENT DES FICHIERS CSV PAR VILLE
# ============================================================================

print("="*80)
print("CHARGEMENT DES FICHIERS CSV PAR VILLE")
print("="*80)

# Dictionnaire des villes avec leurs chemins
villes_data = {
    "Paris": "./villes/Base_OP_2024_L7501.csv",
    "Bordeaux": "./villes/Base_OP_2024_L3300.csv",
    "Grenoble": "./villes/Base_OP_2023_L3800.csv",
    "Lille": "./villes/Base_OP_2023_L5900.csv",
    "Lyon": "./villes/Base_OP_2024_L6900.csv",
    "Marseille": "./villes/Base_OP_2024_L1300.csv",
    "Montpellier": "./villes/Base_OP_2024_L3400.csv",
    "Rennes": "./villes/Base_OP_2024_L3500.csv",
    "Toulouse": "./villes/Base_OP_2024_L3100.csv",
}

# Dictionnaire pour stocker les dataframes bruts
dataframes_bruts = {}
status_chargement = {}

# Chargement avec gestion d'erreurs
for ville, chemin in villes_data.items():
    try:
        df = pd.read_csv(chemin, sep=";", encoding="latin-1")

        # POINT DE CONTRÔLE 1 : Vérifier le chargement brut
        print(f"\n [{ville}] Fichier chargé avec succès")
        dataframes_bruts[ville] = df
        status_chargement[ville] = "SUCCÈS"

    except FileNotFoundError:
        print(f"\n [{ville}] FICHIER NON TROUVÉ")
        print(f"   → Chemin : {chemin}")
        status_chargement[ville] = " FICHIER ABSENT"

    except Exception as e:
        print(f"\n❌ [{ville}] ERREUR DE CHARGEMENT")
        print(f"   → Détail : {str(e)[:100]}")
        status_chargement[ville] = f"❌ ERREUR"

# ============================================================================
# SECTION 2 : RÉCAPITULATIF DU CHARGEMENT
# ============================================================================

print("\n" + "="*80)
print("📊 RÉCAPITULATIF DU CHARGEMENT")
print("="*80)

for ville in sorted(status_chargement.keys()):
    status = status_chargement[ville]
    print(f"{status} - {ville}")

villes_ok = [v for v, s in status_chargement.items() if "SUCCÈS" in s]
print(f"\n✨ Villes chargées avec succès : {len(villes_ok)}/{len(villes_data)}")

# ============================================================================
# SECTION 3 : APERÇU DES DONNÉES BRUTES
# ============================================================================

print("\n" + "="*80)
print("🔍 APERÇU DES DONNÉES BRUTES (5 premières lignes)")
print("="*80)

for ville in villes_ok:
    df = dataframes_bruts[ville]
    print(f"\n📍 {ville.upper()}")
    print(f"   Shape : {df.shape[0]} lignes × {df.shape[1]} colonnes")
    print(f"\n{df.head(5)}")
    print("-" * 80)

# ============================================================================
# SECTION 4 : VÉRIFICATION DES COLONNES IMPORTANTES
# ============================================================================

print("\n" + "="*80)
print("🔎 VÉRIFICATION DES COLONNES DISPONIBLES")
print("="*80)

colonnes_utiles_cibles = [
    "agglomeration", "Data_year", "Type_habitat",
    "loyer_median", "loyer_moyen", "loyer_mensuel_median",
    "surface_moyenne", "nombre_observations"
]

print(f"\nColonnes recherchées : {colonnes_utiles_cibles}\n")

for ville in villes_ok:
    df = dataframes_bruts[ville]
    print(f"📍 {ville}:")

    for col in colonnes_utiles_cibles:
        if col in df.columns:
            print(f"   ✅ {col}")
        else:
            print(f"   ❌ {col} - MANQUANT")

    colonnes_manquantes = [c for c in colonnes_utiles_cibles if c not in df.columns]
    if colonnes_manquantes:
        print(f"   ⚠️  COLONNES MANQUANTES : {colonnes_manquantes}")
    else:
        print(f"   ✓ Toutes les colonnes utiles présentes !")
    print()

print("="*80)
print("✨ CHARGEMENT TERMINÉ - Prêt pour l'étape suivante !")
print("="*80)

# Stocker les dataframes bruts dans une variable globale
globals()['dataframes_bruts'] = dataframes_bruts
globals()['villes_ok'] = villes_ok

print("\n💾 Variables disponibles pour la prochaine étape :")
print("   • dataframes_bruts : dictionnaire des dataframes par ville")
print("   • villes_ok : liste des villes chargées avec succès")






# Pour chaque ville, vérifier les colonnes
colonnes_presentes = {}

for ville in villes_ok:
    df = dataframes_bruts[ville]
    print(f"📍 {ville}:")

    colonnes_ok = []
    colonnes_manquantes = []

    for col in colonnes_utiles_cibles:
        if col in df.columns:
            print(f"   ✅ {col}")
            colonnes_ok.append(col)
        else:
            print(f"   ❌ {col} - MANQUANT")
            colonnes_manquantes.append(col)

    colonnes_presentes[ville] = {
        "ok": colonnes_ok,
        "manquantes": colonnes_manquantes
    }

    if colonnes_manquantes:
        print(f"   ⚠️  Colonnes manquantes : {colonnes_manquantes}")
    else:
        print(f"   ✓ Toutes les colonnes utiles présentes !")
    print()

# ============================================================================
# SECTION 2 : SÉLECTION DES COLONNES UTILES PAR VILLE
# ============================================================================

print("\n" + "="*80)
print("✂️ SÉLECTION DES COLONNES UTILES")
print("="*80)

dataframes_clean = {}

for ville in villes_ok:
    df = dataframes_bruts[ville]

    # Sélectionner uniquement les colonnes qui existent
    colonnes_presentes_ville = colonnes_presentes[ville]["ok"]
    df_clean = df[colonnes_presentes_ville].copy()

    # Ajouter colonne Ville
    df_clean["Ville"] = ville

    dataframes_clean[ville] = df_clean

    print(f"\n📍 {ville}")
    print(f"   Colonnes avant : {df.shape[1]}")
    print(f"   Colonnes après sélection : {df_clean.shape[1]}")
    print(f"   Colonnes finales : {list(df_clean.columns)}")

# ============================================================================
# SECTION 3 : AFFICHAGE DES 5 PREMIÈRES LIGNES NETTOYÉES
# ============================================================================

print("\n" + "="*80)
print("📋 APERÇU DES DONNÉES NETTOYÉES (5 premières lignes)")
print("="*80)

for ville in villes_ok:
    df_clean = dataframes_clean[ville]
    print(f"\n📍 {ville.upper()}")
    print(f"   Shape : {df_clean.shape[0]} lignes × {df_clean.shape[1]} colonnes")
    print(f"\n{df_clean.head(5).to_string()}")
    print("-" * 80)

print("\n" + "="*80)
print("✨ NETTOYAGE TERMINÉ")
print("="*80)

print("\n✅ Prêt pour l'étape suivante : Fusion des données !")


# ============================================================================
# SECTION 4 : FUSION ET EXPORT EN CSV AVEC NOM DE VILLE
# ============================================================================

print("\n" + "="*80)
print("💾 EXPORT DES DONNÉES NETTOYÉES (avec noms de villes)")
print("="*80)

# Fusionner tous les DataFrames des villes
df_final = pd.concat(dataframes_clean.values(), ignore_index=True)

# Supprimer les colonnes inutiles si elles existent
colonnes_a_supprimer = ["agglomeration", "Zone_calcul", "Zone_complementaire"]
for col in colonnes_a_supprimer:
    if col in df_final.columns:
        df_final.drop(columns=col, inplace=True, errors="ignore")

# Nettoyer le nom de la ville (éviter les codes comme L7501)
# Si ta colonne "Ville" contient déjà le vrai nom, tu peux ignorer cette partie.
# Sinon, on peut remplacer manuellement les codes :
mapping_villes = {
    "L7501": "Paris",
    "L3300": "Bordeaux",
    "L6900": "Lyon",
    "L3100": "Toulouse",
    "L5900": "Lille",
    "L6700": "Strasbourg",
    "L3500": "Rennes",
    "L4400": "Nantes",
    "L3400": "Montpellier",
    "L3800": "Grenoble"
}

df_final["Ville"] = df_final["Ville"].replace(mapping_villes)

# Sauvegarder le fichier final
nom_fichier = "loyers_villes.csv"
df_final.to_csv(nom_fichier, index=False, encoding="utf-8-sig")

print(f"\n✅ Données nettoyées et exportées dans : {nom_fichier}")
print(f"Shape finale : {df_final.shape[0]} lignes × {df_final.shape[1]} colonnes")

# Vérifier les premières lignes
print("\nAperçu du DataFrame final :")
print(df_final.head(10))

